In [3]:
import sqlite3

conn = sqlite3.connect("badge_attendance.db")
cursor = conn.cursor()


In [1]:
%load_ext sql

In [2]:
%sql sqlite:///badge_attendance.db

In [6]:
%%sql
CREATE TABLE IF NOT EXISTS fact_attendance_daily (
    badge_id TEXT,
    date_id INTEGER,

    total_hours REAL,
    total_sessions INTEGER,

    first_in_time TEXT,
    last_out_time TEXT,

    avg_session_hours REAL,

    PRIMARY KEY (badge_id, date_id)
);


 * sqlite:///badge_attendance.db
Done.


[]

In [7]:
%%sql
INSERT INTO fact_attendance_daily
SELECT
    e.badge_id,
    c.date_id,

    SUM(
        (julianday(out_time) - julianday(in_time)) * 24
    ) AS total_hours,

    COUNT(*) AS total_sessions,

    MIN(in_time) AS first_in_time,
    MAX(out_time) AS last_out_time,

    AVG(
        (julianday(out_time) - julianday(in_time)) * 24
    ) AS avg_session_hours

FROM (
    SELECT
        badge_id,
        DATE(event_time) AS event_date,

        MIN(CASE WHEN event_type = 'IN' THEN event_time END) AS in_time,
        MAX(CASE WHEN event_type = 'OUT' THEN event_time END) AS out_time

    FROM src_badge_events
    GROUP BY badge_id, DATE(event_time)
) e

JOIN dim_calendar c
ON e.event_date = c.full_date

GROUP BY e.badge_id, c.date_id;


 * sqlite:///badge_attendance.db
301570 rows affected.


[]

In [8]:
%%sql
SELECT COUNT(*) FROM fact_attendance_daily;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
301570


In [9]:
%%sql
SELECT * FROM fact_attendance_daily LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,date_id,total_hours,total_sessions,first_in_time,last_out_time,avg_session_hours
BADGE000000,20240901,3.58333333209157,1,2024-09-01T10:25:00,2024-09-01T14:00:00,3.58333333209157
BADGE000000,20240903,4.966666661202908,1,2024-09-03T11:12:00,2024-09-03T16:10:00,4.966666661202908
BADGE000000,20240904,3.8833333365619183,1,2024-09-04T14:05:00,2024-09-04T17:58:00,3.8833333365619183
BADGE000000,20240905,7.516666665673256,1,2024-09-05T08:49:00,2024-09-05T16:20:00,7.516666665673256
BADGE000000,20240906,4.766666661947966,1,2024-09-06T12:53:00,2024-09-06T17:39:00,4.766666661947966
BADGE000000,20240908,1.2500000037252903,1,2024-09-08T13:44:00,2024-09-08T14:59:00,1.2500000037252903
BADGE000000,20240909,3.4666666723787785,1,2024-09-09T12:48:00,2024-09-09T16:16:00,3.4666666723787785
BADGE000000,20240910,2.9500000029802322,1,2024-09-10T12:47:00,2024-09-10T15:44:00,2.9500000029802322
BADGE000000,20240912,6.683333326131105,1,2024-09-12T08:12:00,2024-09-12T14:53:00,6.683333326131105
BADGE000000,20240914,3.8833333365619183,1,2024-09-14T11:09:00,2024-09-14T15:02:00,3.8833333365619183


In [10]:
%%sql
SELECT
    f.badge_id,
    c.full_date,
    f.total_hours
FROM fact_attendance_daily f
JOIN dim_calendar c
    ON f.date_id = c.date_id
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


badge_id,full_date,total_hours
BADGE000000,2024-09-01,3.58333333209157
BADGE000000,2024-09-03,4.966666661202908
BADGE000000,2024-09-04,3.8833333365619183
BADGE000000,2024-09-05,7.516666665673256
BADGE000000,2024-09-06,4.766666661947966
BADGE000000,2024-09-08,1.2500000037252903
BADGE000000,2024-09-09,3.4666666723787785
BADGE000000,2024-09-10,2.9500000029802322
BADGE000000,2024-09-12,6.683333326131105
BADGE000000,2024-09-14,3.8833333365619183


In [11]:
%%sql
SELECT
    s.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_students s
    ON f.badge_id = s.badge_id
GROUP BY s.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Stephen Smith,1122.4333333969116
James Davis,1108.0333333611488
Jeffrey Mitchell,1064.3166668191552
Daniel Scott,1048.300000011921
Patricia King,1045.1833333075047
Amanda Rodriguez,1037.0333332903683
Jennifer Miller,1031.7333332933486
Nicholas Smith,1022.81666668877
Jennifer Smith,1013.0333332456648
Michael King,1010.849999986589


In [12]:
%%sql
SELECT COUNT(*)
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id;


 * sqlite:///badge_attendance.db
Done.


COUNT(*)
29551


In [13]:
%%sql
SELECT
    t.name AS teacher_name,
    t.department AS subject,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t
ON f.badge_id = t.badge_id
GROUP BY t.name, t.department
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


teacher_name,subject,total_hours
Crystal Williams,Science,1148.6333333328366
Gina Pearson,Engineering,1139.1666667498648
Kathleen Savage,Law,1121.0833332762122
Shane Martin,Arts,1119.3833333216608
Tyler Bush,Engineering,1109.366666637361
Alejandro Baker,Law,1100.583333324641
Christopher Young,Business,1100.2333333343267
Elizabeth Page,Law,1097.8999999389052
Daniel Gates,Science,1097.6666666977108
Mark Hernandez,Engineering,1097.1000000312924


In [14]:
%%sql
SELECT
    t.name,
    SUM(f.total_hours) AS total_hours
FROM fact_attendance_daily f
JOIN dim_teachers t ON f.badge_id = t.badge_id
JOIN dim_calendar c ON f.date_id = c.date_id
WHERE c.year = 2024 AND c.month = 9
GROUP BY t.name
ORDER BY total_hours DESC
LIMIT 10;


 * sqlite:///badge_attendance.db
Done.


name,total_hours
Stephanie Pope,181.28333334624767
Gina Pearson,179.33333333954215
Kathleen Savage,177.66666669398546
Crystal Williams,172.61666668206453
Kendra Grimes,172.56666665151715
Jessica Hoover,170.93333334848285
Mary Thomas,168.9999999627471
Christopher Young,168.98333331942558
Jeremy Foster,166.54999995231628
Danielle Lamb,165.73333332315087
